# Segmentação Avançada com Remoção de Artefatos

Este notebook adiciona remoção automática de pontos pretos (marcas de medição/artefatos) antes da segmentação de microplásticos.

## Novos Recursos:

### 1. Remoção de Pontos Pretos:
- **Detecta artefatos**: Pontos muito escuros (< 30 intensidade)
- **Filtra por tamanho**: Remove apenas pontos pequenos
- **Preserva filamentos**: Mantém objetos escuros grandes
- **Inpainting**: Preenche áreas removidas com cores vizinhas

### 2. Algoritmo de Remoção:
```python
# 1. Detecta pixels muito escuros
black_dots = gray < 30

# 2. Remove objetos grandes (filamentos legítimos)
eroded = cv2.erode(black_dots, kernel_large)
small_dots = cv2.dilate(eroded, kernel_large)

# 3. Preenche pontos pequenos com inpainting
cleaned = cv2.inpaint(image, small_dots, 3, cv2.INPAINT_TELEA)
```

### 3. Benefícios:
- **Segmentação mais limpa**: Remove interferência de marcas
- **Contagem precisa**: Evita falsos positivos
- **Preserva dados**: Mantém microplásticos legítimos
- **Controle opcional**: Pode ser desabilitado se necessário

In [1]:
# Importação das bibliotecas necessárias
import cv2  # OpenCV para processamento de imagens
import numpy as np  # NumPy para operações com arrays
import matplotlib.pyplot as plt  # Matplotlib para visualização
import os  # Operações do sistema operacional
from ipywidgets import interact, IntSlider, Dropdown, Checkbox  # Widgets interativos
from IPython.display import display  # Display para Jupyter

In [2]:
# Carregamento das imagens disponíveis
data_dir = '../data'  # Diretório das imagens
supported_formats = ('.jpg', '.jpeg', '.png', '.tiff', '.tif')  # Formatos suportados

image_files = []  # Lista de arquivos de imagem
if os.path.exists(data_dir):  # Verifica se diretório existe
    # Filtra apenas arquivos com extensões suportadas
    image_files = [f for f in os.listdir(data_dir) 
                   if f.lower().endswith(supported_formats)]

if not image_files:
    print("Nenhuma imagem encontrada no diretório de dados")
else:
    print(f"Encontradas {len(image_files)} imagens: {image_files}")

Encontradas 26 imagens: ['P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'PCB1.1_LK_Q5.jpg', 'P1A_T1_LD_Q5_100X.jpg', 'CNN_P1_L1_T1_M1_LG_Q8_150x.jpg', 'PCB1.1_LK_Q7.jpg', 'NIVEL 2 (BR1_1CAMP_LO_Q6_125X).jpg', 'PCB1.1_LM_.jpg', 'NIVEL 5 (P1D_T1_LI_Q11_63X).jpg', 'NIVEL 4 (P1D_T2_LJ_Q10_150X).jpg', 'P1A_T1_LC_Q6_100X.jpg', 'CNN_P1_L1_T1_M1_LH_Q3_63x.jpg', 'CNN_P1_L1_T1_M1_LI_Q2_50x.jpg', 'P1A_T1_LG_Q8_150X (1).jpg', 'PCB1.1.jpg', 'NIVEL 1 (BR3_1CAMP_LC_Q6_100X).jpg', 'NIVEL 3 (P1D_T3_LH_Q8_100X (2)).jpg', 'PCB1.1LB_Q9.jpg', 'CNN_P1_L1_T1_M1_LG_Q5_80x.jpg', 'PCB1.1_LK_Q10.jpg', 'CNN_P1_L1_T1_M1_LH_Q9_F1_150x.jpg', 'PCB1.1_LB_Q6.jpg', 'CNN_P1_L1_T1_M1_LC_Q9_100x.jpg', 'CNN_P1_L1_T1_M1_LG_Q11_100x.jpg', 'CNN_P1_L1_T1_M1_LI_Q6_125x.jpg', 'P1A_T1_LI_Q11_40X.jpg']


In [3]:
def segment_microplastics_advanced(image_path, background_threshold=160, min_saturation=20, 
                                 remove_black_dots=True, apply_morphology=True):
    """
    Segmentação avançada com remoção de pontos pretos (artefatos)
    
    Parâmetros:
    - image_path: caminho para a imagem
    - background_threshold: limiar para detectar fundo claro (0-255)
    - min_saturation: saturação mínima para considerar colorido (0-255)
    - remove_black_dots: remover pontos pretos pequenos (marcas/artefatos)
    - apply_morphology: aplicar limpeza morfológica
    
    Retorna:
    - original: imagem original RGB
    - cleaned: imagem com pontos pretos removidos
    - masks: dicionário com máscaras para cada cor
    - segmentations: dicionário com segmentações por cor
    - counts: dicionário com contagem de objetos por cor
    """
    # Carrega e converte imagem para RGB
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Cria cópia para processamento
    processed_img = img_rgb.copy()
    
    if remove_black_dots:
        # Remove pontos pretos pequenos (marcas de medição/artefatos)
        # Converte para escala de cinza para detectar pontos muito escuros
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        # Detecta pixels muito escuros (pontos pretos)
        black_dots_mask = gray < 30  # Limiar muito baixo para pontos pretos
        
        # Aplica operação morfológica para manter apenas pontos pequenos
        # Remove objetos grandes (filamentos escuros legítimos)
        kernel_small = np.ones((3,3), np.uint8)
        kernel_large = np.ones((15,15), np.uint8)
        
        # Erosão com kernel grande remove objetos grandes
        eroded = cv2.erode(black_dots_mask.astype(np.uint8), kernel_large)
        # Dilatação restaura pontos pequenos
        small_dots_mask = cv2.dilate(eroded, kernel_large)
        
        # Aplica inpainting para remover os pontos pequenos
        # cv2.inpaint preenche áreas marcadas com cores vizinhas
        processed_img = cv2.inpaint(processed_img, small_dots_mask, 3, cv2.INPAINT_TELEA)
    
    # Converte imagem processada para HSV
    hsv = cv2.cvtColor(processed_img, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv)
    
    # Separa canais RGB para detecção de fundo
    red_channel = processed_img[:,:,0]
    green_channel = processed_img[:,:,1]
    blue_channel = processed_img[:,:,2]
    
    # Detecta fundo claro usando média dos canais RGB
    background_mask = ((red_channel.astype(np.float32) + 
                       green_channel.astype(np.float32) + 
                       blue_channel.astype(np.float32)) / 3) > background_threshold
    
    # Inverte para focar em objetos (não-fundo)
    object_mask = ~background_mask
    
    # Detecta cores usando HSV (mesmo algoritmo da versão anterior)
    blue_mask = object_mask & (h >= 100) & (h <= 130) & (s > min_saturation) & (v > 15)
    red_mask = object_mask & (((h >= 0) & (h <= 25)) | (h >= 170)) & (s > min_saturation) & (v > 20)
    green_mask = object_mask & (h >= 40) & (h <= 80) & (s > min_saturation) & (v > 20)
    yellow_mask = object_mask & (h >= 25) & (h <= 40) & (s > min_saturation) & (v > 30)
    
    # Detecta objetos escuros restantes (mais restritivo após remoção de pontos)
    dark_mask = object_mask & (v < 60) & (s < min_saturation)
    dark_mask = dark_mask & ~blue_mask & ~red_mask & ~green_mask & ~yellow_mask
    
    # Converte máscaras para uint8
    masks = {
        'azul': (blue_mask * 255).astype(np.uint8),
        'vermelho': (red_mask * 255).astype(np.uint8),
        'verde': (green_mask * 255).astype(np.uint8),
        'amarelo': (yellow_mask * 255).astype(np.uint8),
        'escuro': (dark_mask * 255).astype(np.uint8),
        'fundo': (background_mask * 255).astype(np.uint8)
    }
    
    if apply_morphology:
        kernel = np.ones((2,2), np.uint8)
        for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_OPEN, kernel)
            masks[color] = cv2.morphologyEx(masks[color], cv2.MORPH_CLOSE, kernel)
    
    # Cria segmentações
    segmentations = {}
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        segmentations[color] = cv2.bitwise_and(processed_img, processed_img, mask=masks[color])
    
    # Conta objetos
    counts = {}
    min_area = 15
    for color in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']:
        contours, _ = cv2.findContours(masks[color], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        valid_contours = [c for c in contours if cv2.contourArea(c) > min_area]
        counts[color] = len(valid_contours)
    
    return img_rgb, processed_img, masks, segmentations, counts

In [4]:
def interactive_advanced_segmentation(image_name, background_threshold, min_saturation, 
                                    remove_black_dots, apply_morphology):
    """
    Interface interativa para segmentação avançada com remoção de artefatos
    """
    if not image_files:
        print("Nenhuma imagem disponível")
        return
    
    image_path = os.path.join(data_dir, image_name)
    
    # Realiza segmentação avançada
    original, cleaned, masks, segmentations, counts = \
        segment_microplastics_advanced(image_path, background_threshold, min_saturation, 
                                     remove_black_dots, apply_morphology)
    
    # Cria figura com 13 subplots (2 linhas x 7 colunas)
    plt.figure(figsize=(35, 10))
    
    # LINHA SUPERIOR: Original, limpa, fundo e máscaras
    
    # Subplot 1: Imagem original
    plt.subplot(2, 7, 1)
    plt.imshow(original)
    plt.title('Original\n(Com Artefatos)')
    plt.axis('off')
    
    # Subplot 2: Imagem limpa (se remoção ativada)
    plt.subplot(2, 7, 2)
    if remove_black_dots:
        plt.imshow(cleaned)
        plt.title('Limpa\n(Artefatos Removidos)')
    else:
        plt.imshow(original)
        plt.title('Sem Limpeza\n(Remoção Desabilitada)')
    plt.axis('off')
    
    # Subplot 3: Fundo detectado
    plt.subplot(2, 7, 3)
    plt.imshow(masks['fundo'], cmap='gray')
    plt.title('Fundo\nDetectado')
    plt.axis('off')
    
    # Subplot 4-7: Máscaras de cores
    colors_info = [
        ('azul', 'Blues', 'Azul'),
        ('vermelho', 'Reds', 'Vermelho'),
        ('verde', 'Greens', 'Verde'),
        ('amarelo', 'YlOrBr', 'Amarelo')
    ]
    
    for i, (color, cmap, label) in enumerate(colors_info, 4):
        plt.subplot(2, 7, i)
        plt.imshow(masks[color], cmap=cmap)
        plt.title(f'{label}\n({counts[color]} objetos)')
        plt.axis('off')
    
    # LINHA INFERIOR: Máscara escura, segmentações e estatísticas
    
    # Subplot 8: Máscara escura
    plt.subplot(2, 7, 8)
    plt.imshow(masks['escuro'], cmap='gray')
    plt.title(f'Escuro\n({counts["escuro"]} objetos)')
    plt.axis('off')
    
    # Subplot 9-12: Segmentações
    seg_labels = ['Azuis', 'Vermelhos', 'Verdes', 'Amarelos']
    seg_colors = ['azul', 'vermelho', 'verde', 'amarelo']
    
    for i, (color, label) in enumerate(zip(seg_colors, seg_labels), 9):
        plt.subplot(2, 7, i)
        plt.imshow(segmentations[color])
        plt.title(f'Microplásticos\n{label}')
        plt.axis('off')
    
    # Subplot 13: Gráfico comparativo
    plt.subplot(2, 7, 13)
    bar_colors = ['blue', 'red', 'green', 'gold', 'black']
    bar_labels = ['Azul', 'Vermelho', 'Verde', 'Amarelo', 'Escuro']
    values = [counts[c] for c in ['azul', 'vermelho', 'verde', 'amarelo', 'escuro']]
    
    bars = plt.bar(bar_labels, values, color=bar_colors, alpha=0.7)
    plt.title('Contagem\nTotal')
    plt.ylabel('Objetos')
    plt.xticks(rotation=45, fontsize=8)
    
    # Adiciona valores nas barras
    for bar, value in zip(bars, values):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    str(value), ha='center', va='bottom', fontsize=8)
    
    # Título geral
    total_objects = sum(values)
    removal_status = 'ATIVADA' if remove_black_dots else 'DESATIVADA'
    plt.suptitle(f'Segmentação Avançada - Remoção: {removal_status} | Fundo: {background_threshold} | Saturação: {min_saturation} | Total: {total_objects}', 
                fontsize=16)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Widget interativo para segmentação avançada
if image_files:
    interact(interactive_advanced_segmentation,
             # Dropdown para seleção da imagem
             image_name=Dropdown(options=image_files, description='Imagem:'),
             
             # Slider para limiar de fundo
             background_threshold=IntSlider(min=120, max=200, step=10, value=160, 
                                          description='Limiar Fundo:'),
             
             # Slider para saturação mínima
             min_saturation=IntSlider(min=10, max=60, step=5, value=20, 
                                    description='Saturação Mín:'),
             
             # Checkbox para remoção de pontos pretos (NOVO!)
             remove_black_dots=Checkbox(value=True, 
                                      description='Remover Pontos Pretos'),
             
             # Checkbox para limpeza morfológica
             apply_morphology=Checkbox(value=True, 
                                     description='Aplicar Limpeza Morfológica'))
else:
    print("Adicione imagens ao diretório ../data/ para usar a segmentação avançada")

interactive(children=(Dropdown(description='Imagem:', options=('P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'P…

## Comparação: Antes vs Depois da Remoção

### Algoritmo de Remoção de Pontos Pretos:

#### 1. Detecção de Pixels Escuros:
```python
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
black_dots = gray < 30  # Muito escuro
```

#### 2. Filtragem por Tamanho:
```python
# Remove objetos grandes (filamentos legítimos)
eroded = cv2.erode(black_dots, kernel_15x15)
small_dots = cv2.dilate(eroded, kernel_15x15)
```

#### 3. Inpainting (Preenchimento):
```python
# Preenche pontos com cores vizinhas
cleaned = cv2.inpaint(image, small_dots, 3, cv2.INPAINT_TELEA)
```

### Benefícios Observados:

#### ✅ Com Remoção de Pontos:
- **Segmentação mais limpa**: Sem interferência de marcas
- **Contagem precisa**: Elimina falsos positivos
- **Foco nos microplásticos**: Apenas objetos relevantes
- **Melhor visualização**: Imagens mais claras

#### ❌ Sem Remoção de Pontos:
- **Ruído na segmentação**: Pontos pretos interferem
- **Contagem inflada**: Inclui artefatos como objetos
- **Análise comprometida**: Dados menos confiáveis

### Quando Usar:
- **SEMPRE ativar** para imagens com marcas de medição
- **Desativar apenas** se não houver pontos pretos
- **Testar ambos** para comparar resultados

### Parâmetros Recomendados:
- **Limiar de fundo**: 160 (para fundos cinza típicos)
- **Saturação mínima**: 20 (para cores de microscopia)
- **Remoção de pontos**: ATIVADA (padrão)
- **Limpeza morfológica**: ATIVADA (padrão)